Bu maalesef çalışmıyor !!!!!








İlk olarak Toygar_Beyza_v.4.ipynb defterini çalıştırın

In [5]:
import os

class cfg():
    data_path = "/Users/user/Desktop/Algorun_24/data"

    promotion_calculation = "/Users/user/Desktop/Algorun_24/Algorun_24_programs/Algorun_24_Beyza_Toygar/promotion_sales_calculations.csv"

    seed = 42

    holidays_path = os.path.join(data_path, "holidays.csv")
    products_path = os.path.join(data_path, "products.csv")
    promotions_path = os.path.join(data_path, "promotions.csv")
    sample_submission_path = os.path.join(data_path, "sample_submission.csv")
    test_path = os.path.join(data_path, "test.csv")
    train_path = os.path.join(data_path, "train.csv")

    


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


holidays_df = pd.read_csv(cfg.holidays_path)
products_df = pd.read_csv(cfg.products_path)
promotions_df = pd.read_csv(cfg.promotions_path)
sample_submission_df = pd.read_csv(cfg.sample_submission_path, delimiter='|')
test_df = pd.read_csv(cfg.test_path)
train_df = pd.read_csv(cfg.train_path)

# Convert columns to datetime format

train_df["week_starting_date"] = pd.to_datetime(train_df["week_starting_date"])
test_df["week_starting_date"] = pd.to_datetime(test_df["week_starting_date"])
sample_submission_df["week_starting_date"] = pd.to_datetime(sample_submission_df["week_starting_date"])

In [6]:
train_df["week_starting_date"].max()

train_df = train_df.merge(products_df, on="product_id", how="left")

last_train_values_df = train_df[train_df["week_starting_date"] == train_df["week_starting_date"].max()]
last_train_values_df = last_train_values_df[["product_id", "sales_quantity"]]

second_submission_df = sample_submission_df.copy()

second_submission_df = second_submission_df.merge(last_train_values_df, on="product_id", how="left")

second_submission_df["prediction"] = second_submission_df["sales_quantity"]

second_submission_df = second_submission_df[["product_id", "week_starting_date", "prediction"]]

# Making negative numbers in prediction equal to 0
second_submission_df.loc[second_submission_df["prediction"] < 0, "prediction"] = 0

second_submission_df.fillna(0,inplace=True)

second_submission_df.to_csv('second_submission.csv', index=False,sep="|")

In [7]:
# Fourth submission based on the sale of product groups(group_1 and group_2) during last year same_time, and ratio of sale of products in the last 2 months

# Create a copy of the training data with shifted dates
shifted_train_df = train_df.copy()
shifted_train_df["week_starting_date"] = train_df["week_starting_date"] + dt.timedelta(days=371)

# Combine product group codes into a single string column
shifted_train_df['product_group_1_2_code'] = shifted_train_df['product_group_1_code'].astype(str) + '_' + shifted_train_df['product_group_2_code'].astype(str)

important_dates1 = list(test_df["week_starting_date"].unique())
prepared_shifted_train_df = shifted_train_df[
    shifted_train_df["week_starting_date"].isin(important_dates1)
][["week_starting_date", "product_id", "sales_quantity", "product_group_1_2_code"]]

# Group by week_starting_date and combined product group code
grouped_sales = prepared_shifted_train_df.groupby(
    ['week_starting_date', 'product_group_1_2_code'])['sales_quantity'].sum().reset_index()

# Calculate ratios for the last 2 months
last_date = train_df['week_starting_date'].max()
two_months_ago = last_date - pd.DateOffset(months=2)
last_2months_df = train_df[train_df['week_starting_date'] >= two_months_ago]

# Add the combined product group code to the last_2months_df
last_2months_df['product_group_1_2_code'] = last_2months_df['product_group_1_code'].astype(str) + '_' + last_2months_df['product_group_2_code'].astype(str)

# Calculate ratios with the combined group code
product_sales = last_2months_df.groupby('product_id')['sales_quantity'].sum()
group_sales = last_2months_df.groupby('product_group_1_2_code')['sales_quantity'].sum()

sales_ratio = (
    last_2months_df[['product_id', 'product_group_1_2_code']]
    .drop_duplicates()
    .set_index('product_id')
)

sales_ratio['product_sales'] = product_sales
sales_ratio['group_total_sales'] = sales_ratio['product_group_1_2_code'].map(group_sales)
sales_ratio['sales_ratio'] = sales_ratio['product_sales'] / sales_ratio['group_total_sales']
sales_ratio = sales_ratio.reset_index()

# Merge the data using the combined product group code
merged_data = pd.merge(
    grouped_sales[['week_starting_date', 'product_group_1_2_code', 'sales_quantity']],
    sales_ratio[['product_group_1_2_code', 'product_id', 'sales_ratio']],
    on='product_group_1_2_code'
)

# Calculate predicted sales
merged_data['predicted_sales'] = merged_data['sales_quantity'] * merged_data['sales_ratio']

# Create final predictions dataframe
predictions_df = merged_data[['week_starting_date', 'product_id', 
                            'product_group_1_2_code', 'predicted_sales']]


fourth_submission_df = sample_submission_df.copy()

fourth_submission_df = fourth_submission_df.merge(predictions_df, on=["product_id","week_starting_date"], how="left")

fourth_submission_df["prediction"] = fourth_submission_df["predicted_sales"]

fourth_submission_df = fourth_submission_df[["product_id", "week_starting_date", "prediction"]]

# Making negative numbers in prediction equal to 0
fourth_submission_df.loc[fourth_submission_df["prediction"] < 0, "prediction"] = 0

fourth_submission_df.fillna(0,inplace=True)

fourth_submission_df.to_csv('fourth_submission.csv', index=False,sep="|")

/var/folders/58/pms80m657fs8zklh9996g1br0000gp/T/ipykernel_61053/2377099430.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_2months_df['product_group_1_2_code'] = last_2months_df['product_group_1_code'].astype(str) + '_' + last_2months_df['product_group_2_code'].astype(str)


In [9]:

past_promotions_df = promotions_df[promotions_df["total_sales_quantity"]!=0]

promotion_stats = past_promotions_df.groupby('promotion_type').agg({
    'mean_sales_quantity_per_day': ['mean', 'std', 'min', 'max', 'count'],
    'total_sales_quantity': ['mean', 'std', 'min', 'max']
}).round(2)




def predict_test_sales(test_df, promotions_df, promotion_stats):
    # Print diagnostic information
    print("Test weeks:")
    print(test_df['week_starting_date'].unique())
    
    print("\nPromotions during test period:")
    mask = (promotions_df['start_date'] <= test_df['week_starting_date'].max() + pd.Timedelta(days=6)) & \
           (promotions_df['end_date'] >= test_df['week_starting_date'].min())
    print(promotions_df[mask][['promotion_type', 'start_date', 'end_date']])
    
    print("\nPromotion statistics:")
    print(promotion_stats)
    
    # Get unique week starting dates
    result_df = pd.DataFrame({'week_starting_date': test_df['week_starting_date'].unique()})
    result_df = result_df.sort_values('week_starting_date').reset_index(drop=True)
    result_df['week_end_date'] = result_df['week_starting_date'] + pd.Timedelta(days=6)
    result_df['predicted_sales'] = 0.0
    
    # Get mean daily sales for each promotion type
    mean_daily_sales = promotion_stats['mean_sales_quantity_per_day']['mean']
    
    # For each week, calculate the sales based on active promotions
    for idx, row in result_df.iterrows():
        week_start = row['week_starting_date']
        week_end = row['week_end_date']
        
        # Find promotions active during this week
        active_promos = promotions_df[
            (promotions_df['start_date'] <= week_end) & 
            (promotions_df['end_date'] >= week_start)
        ]
        
        if len(active_promos) > 0:
            for _, promo in active_promos.iterrows():
                # Calculate overlap days
                overlap_start = max(week_start, promo['start_date'])
                overlap_end = min(week_end, promo['end_date'])
                overlap_days = (overlap_end - overlap_start).days + 1
                
                # Add sales for this promotion
                daily_sales = mean_daily_sales[promo['promotion_type']]
                result_df.loc[idx, 'predicted_sales'] += daily_sales * overlap_days
            
            # If multiple promotions, take average
            if len(active_promos) > 1:
                result_df.loc[idx, 'predicted_sales'] /= len(active_promos)
    
    return result_df[['week_starting_date', 'predicted_sales']]

# Generate predictions
predictions = predict_test_sales(test_df, promotions_df, promotion_stats)

# Display results
print("\nFinal Predictions for test weeks:")
print(predictions)

# First calculate the ratios as before
last_date = train_df['week_starting_date'].max()
two_months_ago = last_date - pd.DateOffset(months=2)
last_2months_df = train_df[train_df['week_starting_date'] >= two_months_ago]

# Calculate ratios
product_sales = last_2months_df.groupby('product_id')['sales_quantity'].sum()
total_sales = last_2months_df['sales_quantity'].sum()  # Changed to total sales

sales_ratio = (
    last_2months_df[['product_id']]  # Removed product_group_1_code since we don't need it
    .drop_duplicates()
    .set_index('product_id')
)

sales_ratio['product_sales'] = product_sales
sales_ratio['total_sales'] = total_sales  # Changed to total sales
sales_ratio['sales_ratio'] = sales_ratio['product_sales'] / sales_ratio['total_sales']
sales_ratio = sales_ratio.reset_index()

# Create a cartesian product (cross join) using merge with dummy key
predictions['key'] = 1
sales_ratio['key'] = 1
merged_data = pd.merge(
    predictions[['week_starting_date', 'predicted_sales', 'key']],
    sales_ratio[['product_id', 'sales_ratio', 'key']],
    on='key'
).drop('key', axis=1)

# Vectorized calculation of predicted sales per product
merged_data['predicted_product_sales'] = merged_data['predicted_sales'] * merged_data['sales_ratio']

# Select required columns for final output
predictions_df = merged_data[['week_starting_date', 'product_id', 'predicted_product_sales']]

seventh_submission_df = sample_submission_df.copy()

seventh_submission_df = seventh_submission_df.merge(predictions_df, on=["product_id","week_starting_date"], how="left")

seventh_submission_df["prediction"] = seventh_submission_df["predicted_product_sales"]

seventh_submission_df = seventh_submission_df[["product_id", "week_starting_date", "prediction"]]

# Making negative numbers in prediction equal to 0
seventh_submission_df.loc[seventh_submission_df["prediction"] < 0, "prediction"] = 0

seventh_submission_df.fillna(0,inplace=True)

seventh_submission_df.to_csv('seventh_submission.csv', index=False,sep="|")


KeyError: 'total_sales_quantity'

In [ ]:
# Thirteenth submission based on the sale of product groups(group_1_2_3) during last year same_time, and ratio of sale of products in the last 2 months

# Assuming your DataFrame 'df' has columns: 'product_id', 'product_group', 'sale_date', 'sale_amount'

shifted_train_df = train_df.copy()
shifted_train_df["week_starting_date"] = train_df["week_starting_date"] + dt.timedelta(days=371)

important_dates1 = list(test_df["week_starting_date"].unique())
prepared_shifted_train_df = shifted_train_df[
    shifted_train_df["week_starting_date"].isin(important_dates1)
][["week_starting_date", "product_id", "sales_quantity", "product_group_1_2_3_code"]]

# Group by week_starting_date and product_group_1_2_3_code and sum the sales_quantity
grouped_sales = prepared_shifted_train_df.groupby(
    ['week_starting_date', 'product_group_1_2_3_code'])['sales_quantity'].sum().reset_index()

# If you want to reshape the data to have product groups as columns
# pivoted_sales = grouped_sales.pivot(
#     index='week_starting_date', 
#     columns='product_group_1_2_3_code', 
#     values='sales_quantity'
# )


# First calculate the ratios as before
last_date = train_df['week_starting_date'].max()
two_months_ago = last_date - pd.DateOffset(months=2)
last_2months_df = train_df[train_df['week_starting_date'] >= two_months_ago]

# Calculate ratios
product_sales = last_2months_df.groupby('product_id')['sales_quantity'].sum()
group_sales = last_2months_df.groupby('product_group_1_2_3_code')['sales_quantity'].sum()

sales_ratio = (
    last_2months_df[['product_id', 'product_group_1_2_3_code']]
    .drop_duplicates()
    .set_index('product_id')
)

sales_ratio['product_sales'] = product_sales
sales_ratio['group_total_sales'] = sales_ratio['product_group_1_2_3_code'].map(group_sales)
sales_ratio['sales_ratio'] = sales_ratio['product_sales'] / sales_ratio['group_total_sales']
sales_ratio = sales_ratio.reset_index()



# Perform a merge operation instead of iterating
merged_data = pd.merge(
    grouped_sales[['week_starting_date', 'product_group_1_2_3_code', 'sales_quantity']],
    sales_ratio[['product_group_1_2_3_code', 'product_id', 'sales_ratio']],
    on='product_group_1_2_3_code'
)

# Vectorized calculation of predicted sales
merged_data['predicted_sales'] = merged_data['sales_quantity'] * merged_data['sales_ratio']

# Select required columns for final output
predictions_df = merged_data[['week_starting_date', 'product_id', 'predicted_sales']]

thirteenth_submission_df = sample_submission_df.copy()

thirteenth_submission_df = thirteenth_submission_df.merge(predictions_df, on=["product_id","week_starting_date"], how="left")

thirteenth_submission_df["prediction"] = thirteenth_submission_df["predicted_sales"]

thirteenth_submission_df = thirteenth_submission_df[["product_id", "week_starting_date", "prediction"]]

# Making negative numbers in prediction equal to 0
thirteenth_submission_df.loc[thirteenth_submission_df["prediction"] < 0, "prediction"] = 0


thirteenth_submission_df = sample_submission_df.copy()

# First merge with predictions
thirteenth_submission_df = thirteenth_submission_df.merge(predictions_df, on=["product_id","week_starting_date"], how="left")

# Merge with product information to get product_group_1_2_3_code
thirteenth_submission_df = thirteenth_submission_df.merge(
    train_df[['product_id', 'product_group_1_2_3_code']].drop_duplicates(),
    on='product_id',
    how='left'
)

# Fill NaN values with mean predictions for the same product group and week
thirteenth_submission_df['predicted_sales'] = thirteenth_submission_df.groupby(
    ['product_group_1_2_3_code', 'week_starting_date'])['predicted_sales'].transform(
    lambda x: x.fillna(x.mean())
)

thirteenth_submission_df["prediction"] = thirteenth_submission_df["predicted_sales"]



thirteenth_submission_df.fillna(3.5, inplace=True)


# Keep only required columns
thirteenth_submission_df = thirteenth_submission_df[["product_id", "week_starting_date", "prediction"]]

# Making negative numbers in prediction equal to 0
thirteenth_submission_df.loc[thirteenth_submission_df["prediction"] < 0, "prediction"] = 0

thirteenth_submission_df.isna().sum()

thirteenth_submission_df.to_csv('thirteenth_submission.csv', index=False,sep="|")



KeyError: "['product_group_1_2_3_code'] not in index"

In [ ]:
# Fourteenth submission will be based on thirteenth submission, but the predictions will be multiplied with (thirteenth submission total sales + promotion calculation total sales)/ (2*thirteenth submission total sales)

# Read thirteenth submission
thirteenth_submission_df = pd.read_csv('thirteenth_submission.csv', delimiter='|')
thirteenth_submission_df['week_starting_date'] = pd.to_datetime(thirteenth_submission_df['week_starting_date'])

# Calculate total sales from thirteenth submission
thirteenth_total_sales = thirteenth_submission_df['prediction'].sum()

# Calculate promotion-based predictions
def predict_test_sales(test_df, promotions_df, promotion_stats):
    result_df = pd.DataFrame({'week_starting_date': test_df['week_starting_date'].unique()})
    result_df = result_df.sort_values('week_starting_date').reset_index(drop=True)
    result_df['week_end_date'] = result_df['week_starting_date'] + pd.Timedelta(days=6)
    result_df['predicted_sales'] = 0.0
    
    mean_daily_sales = promotion_stats['mean_sales_quantity_per_day']['mean']
    
    for idx, row in result_df.iterrows():
        week_start = row['week_starting_date']
        week_end = row['week_end_date']
        
        active_promos = promotions_df[
            (promotions_df['start_date'] <= week_end) & 
            (promotions_df['end_date'] >= week_start)
        ]
        
        if len(active_promos) > 0:
            for _, promo in active_promos.iterrows():
                overlap_start = max(week_start, promo['start_date'])
                overlap_end = min(week_end, promo['end_date'])
                overlap_days = (overlap_end - overlap_start).days + 1
                
                daily_sales = mean_daily_sales[promo['promotion_type']]
                result_df.loc[idx, 'predicted_sales'] += daily_sales * overlap_days
            
            if len(active_promos) > 1:
                result_df.loc[idx, 'predicted_sales'] /= len(active_promos)
    
    return result_df[['week_starting_date', 'predicted_sales']]

# Generate promotion-based predictions
promotion_predictions = pd.read_csv(cfg.promotion_calculation,delimiter="|")
promotion_total_sales = promotion_predictions['prediction'].sum()

# Calculate adjustment factor
adjustment_factor = (thirteenth_total_sales + promotion_total_sales) / (2 * thirteenth_total_sales)

# Create fourteenth submission
fourteenth_submission_df = thirteenth_submission_df.copy()
fourteenth_submission_df['prediction'] = fourteenth_submission_df['prediction'] * adjustment_factor

# Ensure no negative predictions
fourteenth_submission_df.loc[fourteenth_submission_df['prediction'] < 0, 'prediction'] = 0

# Round predictions to desired decimal places (optional)
fourteenth_submission_df['prediction'] = fourteenth_submission_df['prediction'].round(2)

# Save the results
fourteenth_submission_df.to_csv('fourteenth_submission.csv', sep='|', index=False)

# Print summary statistics
print(f"Thirteenth submission total sales: {thirteenth_total_sales:,.2f}")
print(f"Promotion-based total sales: {promotion_total_sales:,.2f}")
print(f"Adjustment factor: {adjustment_factor:.4f}")
print(f"Fourteenth submission total sales: {fourteenth_submission_df['prediction'].sum():,.2f}")

In [ ]:
# Fifteenth submission is based on eighth submission, but instead of filling NaN with 0, I will be filling with the value that minimizes RMSLE in train_df

import pandas as pd
second_submission_df = pd.read_csv('second_submission.csv',delimiter="|")
fourth_submission_df = pd.read_csv('fourth_submission.csv',delimiter="|")
fourteenth_submission_df = pd.read_csv('fourteenth_submission.csv', delimiter="|")

eighth_submission_df2 = second_submission_df.copy()
eighth_submission_df2["prediction"] = (8 * second_submission_df["prediction"] + fourth_submission_df["prediction"] + fourteenth_submission_df["prediction"]) / 10

# Making negative numbers in prediction equal to 0
eighth_submission_df2.loc[eighth_submission_df2["prediction"] < 0, "prediction"] = 0

eighth_submission_df2.fillna(0,inplace=True)

eighth_submission_df2.to_csv('eighth_submission.csv', index=False, sep="|")




fifteenth_submission_df = eighth_submission_df2.copy()
fifteenth_submission_df['prediction'] = fifteenth_submission_df['prediction'].fillna(0)

# Ensure no negative predictions
fifteenth_submission_df.loc[fifteenth_submission_df['prediction'] < 0, 'prediction'] = 0

# Keep only required columns
fifteenth_submission_df = fifteenth_submission_df[['product_id', 'week_starting_date', 'prediction']]

# Save the results
fifteenth_submission_df.to_csv('fifteenth_submission.csv', sep='|', index=False)

fifteenth_submission_df.describe()

In [ ]:
# Sixteenth submission is based on a new ensemble

second_submission_df = pd.read_csv('second_submission.csv',delimiter="|")
fourth_submission_df = pd.read_csv('fourth_submission.csv',delimiter="|")
seventh_submission_df = pd.read_csv('seventh_submission.csv', delimiter="|")
fourteenth_submission_df = pd.read_csv('fourteenth_submission.csv', delimiter="|")


sixteenth_submission_df = sample_submission_df.copy()
sixteenth_submission_df["prediction"] = (2 * second_submission_df["prediction"] + fourth_submission_df["prediction"] + seventh_submission_df["prediction"] + fourteenth_submission_df["prediction"]) / 5
sixteenth_submission_df['prediction'] = sixteenth_submission_df['prediction'].fillna(0)

# Ensure no negative predictions
sixteenth_submission_df.loc[sixteenth_submission_df['prediction'] < 0, 'prediction'] = 0

sixteenth_submission_df['prediction'] = sixteenth_submission_df['prediction']*(16.947841/17.474522)

# Keep only required columns
sixteenth_submission_df = sixteenth_submission_df[['product_id', 'week_starting_date', 'prediction']]

# Save the results
sixteenth_submission_df.to_csv('sixteenth_submission.csv', sep='|', index=False)

sixteenth_submission_df.describe()